notes on 2020.2.10:
- pysteps result is calculated in a seperate shell. This notebook is only used for case checking

In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
print(os.getcwd())

from rainymotion import models, metrics, utils
from collections import OrderedDict
import h5py
import wradlib.ipol as ipol
from datetime import datetime,timedelta
import pandas as pd
import joblib
data_folder = "/Users/jiang/data/jma_radar"
daily_folder = "/Users/jiang/data/jma_radar/2019_daily_unit8"
csv_path = os.path.join(data_folder,"threat_coverage.csv")
data0 = pd.read_csv(csv_path)

/Users/jiang/github_all/rainymotion/docs/notebooks


In [4]:
np.random.seed(3)  # only valid for once
select = np.random.choice(23508, 100).astype(np.int16)
select.sort()
print(data0.loc[select, "threat"].median())

0.5378013431239785


## rainymotion's results

In [14]:
%%time

time_step = 5 * 60 # seconds
threshold = 0.1  # mm/h

modela = models.Dense60()  
# default method is "DIS"
# default advection is "constant-vector"
daily_database = {}
inputs = np.zeros(shape = (2,1000,1000), dtype = np.float16)
a_threat = []
per_threat = []
for i in select:  
    dt =     datetime.strptime(data0.loc[i,"datetime"], '%Y-%m-%d %H:%M:%S')
    dt_now = dt + timedelta(seconds = time_step)
    dt_12  = dt + timedelta(seconds = time_step * 13)
    yday_str = dt.strftime('%Y_%m_%d')    
    yday_now = dt_now.strftime('%Y_%m_%d')
    yday_12 =  dt_12.strftime('%Y_%m_%d')

    if yday_str not in daily_database:
        daily_database[yday_str] = joblib.load(os.path.join(daily_folder, f"jma_radar_uint8_{yday_str}.joblib"))
    if yday_12 not in daily_database:
        daily_database[yday_12] =  joblib.load(os.path.join(daily_folder, f"jma_radar_uint8_{yday_12}.joblib")) 

    inputs[0,:,:] = daily_database[yday_str][int(dt.hour*12     + (dt.minute)/5)]/10.0   # back to original scale
    inputs[1,:,:] = daily_database[yday_now][int(dt_now.hour*12 + (dt_now.minute)/5)]/10.0
    gt_in_60_min =   daily_database[yday_12][int(dt_12.hour*12  + (dt_12.minute)/5)]/10.0
    modela.input_data = inputs
    a_threat.append(metrics.CSI(gt_in_60_min, modela.run(), threshold = threshold))
    per_threat.append(metrics.CSI(gt_in_60_min, inputs[1,:,:], threshold = threshold)) 
# Wall time: 1 min
data0.loc[select, "threat"].median(), np.median(a_threat), np.median(per_threat)

CPU times: user 5min 7s, sys: 24.7 s, total: 5min 32s
Wall time: 1min 23s


(0.5378013431239785, 0.5391066629046685, 0.473375868031846)

## Pysteps's OF + rainymotion's advection

In [21]:
%%time

time_step = 5 * 60 # seconds
threshold = 0.1  # mm/h

all_uv = joblib.load("/Users/jiang/data/all_uv_vet.joblib")

daily_database = {}
#inputs = np.zeros(shape = (3,1000,1000), dtype = np.float16)
inputs = np.zeros(shape = (1000,1000), dtype = np.float16)
threshold = 0.1
zerovalue = -15.0

step_threat = []
cnt = -1
for i in select:  
    cnt += 1 
    dt =     datetime.strptime(data0.loc[i,"datetime"], '%Y-%m-%d %H:%M:%S')
    dt_minus = dt - timedelta(seconds = time_step)
    dt_now = dt + timedelta(seconds = time_step)
    dt_12  = dt + timedelta(seconds = time_step * 13)
    yday_minus = dt_minus.strftime('%Y_%m_%d')
    yday_str = dt.strftime('%Y_%m_%d')    
    yday_now = dt_now.strftime('%Y_%m_%d')
    yday_12 =  dt_12.strftime('%Y_%m_%d')

    if yday_minus not in daily_database:
        daily_database[yday_minus] = joblib.load(os.path.join(daily_folder, f"jma_radar_uint8_{yday_minus}.joblib")) 
    if yday_12 not in daily_database:
        daily_database[yday_12] =  joblib.load(os.path.join(daily_folder, f"jma_radar_uint8_{yday_12}.joblib")) 
    
    sequence_in_day = int(dt_minus.hour*12 + (dt_minus.minute)/5)    
    #inputs[0,:,:] = daily_database[yday_minus][sequence_in_day]/10.0
    #inputs[1,:,:] = daily_database[yday_str][(sequence_in_day + 1)%288]/10.0
    inputs = daily_database[yday_now][(sequence_in_day + 2)%288]/10.0
    #zeros = inputs < threshold
    #inputs[~zeros] = 10.0 * np.log10(inputs [~zeros] )
    #inputs[zeros] = zerovalue

    gt_in_60_min =   daily_database[yday_12][int(dt_12.hour*12  + (dt_12.minute)/5)]/10.0
    
    uv = all_uv[cnt]
    u = uv[0]
    v = uv[1]    
    coord_source_i, coord_source_j = np.meshgrid(range(u.shape[1]),range(u.shape[0]))
    coord_source = [coord_source_i, coord_source_j]
    # calculate new coordinates of radar pixels
    coord_target_i = coord_source_i + u * 12
    coord_target_j = coord_source_j + v * 12
    coord_targets =[coord_target_i, coord_target_j]
    steps = models._interpolator(inputs, coord_source, coord_targets, method= "idw")
    
    step_threat.append(metrics.CSI(gt_in_60_min, steps, threshold = threshold))

CPU times: user 4min 43s, sys: 18.9 s, total: 5min 2s
Wall time: 1min 12s


## convert to panda dataframe and do data analysis

In [22]:
data = data0.loc[select,:]
data['threat_new']  = a_threat
data['threat_step'] = step_threat
data['threat_per']  = per_threat
np.median(data.threat),np.median(data.threat_new), np.median(data.threat_step), np.median(data.threat_per)

(0.5378013431239785, 0.5391066629046685, 0.5349609769117923, 0.473375868031846)